In [0]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [0]:
Carros_temp = spark.read.csv("/FileStore/tables/Carros.csv",inferSchema=True, header=True, sep=";")
Carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [0]:
Carros = Carros_temp.select("Consumo","Cilindros","Cilindradas","HP")
Carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [0]:
veccaracteristicas = VectorAssembler(inputCols=[("Consumo"),("Cilindros"),("Cilindradas")],outputCol="caracteristicas")
Carros = veccaracteristicas.transform(Carros)
Carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [0]:
CarrosTreino, CarrosTeste = Carros.randomSplit([0.7,0.3]) # treino = 70% / teste = 30%
print(CarrosTreino.count())
print(CarrosTeste.count())

23
9


In [0]:
reglin = LinearRegression(featuresCol="caracteristicas", labelCol="HP")
modelo = reglin.fit(CarrosTreino) # treinando o modelo

In [0]:
previsao = modelo.transform(CarrosTeste)
previsao.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]| 173.2499822726199|
|    143|        8|        360|245| [143.0,8.0,360.0]| 214.3958054670549|
|    152|        8|        304|150| [152.0,8.0,304.0]| 213.5873218253566|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|192.33738818437982|
|    192|        6|       1676|123|[192.0,6.0,1676.0]|135.54778705194138|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



In [0]:
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP",metricName='rmse')
rmse = avaliar.evaluate(previsao)
print(rmse)

34.84906884928061
